<a href="https://colab.research.google.com/github/neelkanthrawat/GNN-exercises/blob/main/Assignment_1_Neel's_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from typing import List
from numpy.linalg import norm

In [2]:
### define a gaussian function in n_star dimn

####

In [3]:
### define two dimensional gaussian distribution

### this gaussian has the ability to infer (evaluate the probability at the given point)
def gaussian_2d(x:np.array,mean:np.array,cov_mat:np.array):
  '''
  This model will evaluate the probabilty at the given x_vector.
  Args:
  1. x= np array of shape (1 X dimn_of_the_problem=2)
  2. mean= np array of shape (1 X dimn_of_the_problem=2)
  3.cov_mat= square np array of shape (dimn_of_the_prob=2)
  '''
  dimn=np.shape(x)[0]
  # x=np.array(x).reshape((1,dimn))
  # mean=np.array(mean).reshape((1,dimn))
  # print("shape(x),shape(mean)")
  # print(np.shape(x), np.shape(mean))
  # note that x is a row vector

  inv_cov=np.linalg.inv(cov_mat)
  x_minus_mu=x-mean

  # argument for the exp
  arg_for_expn=0.5*(x_minus_mu @ inv_cov @ np.transpose(x_minus_mu))[0,0]
  det_cov=np.linalg.det(cov_mat)
  # evaluating the prob
  prob=np.exp(-arg_for_expn) * (1./(2*np.pi*np.sqrt(det_cov)))

  return prob

In [4]:
x=np.array([[0.5,0.5]])
mean=np.array([[1,1]])
cov_mat=np.array([[0.3,0.22],[0.22,0.8]])
check=gaussian_2d(x,mean,cov_mat)
print(check)

0.2363860723405543


### fitting the gaussian model
###  when we fit the model, we need some loss function to optimize to find the parameters, is'nt it

### My model should have 2 functionalities:
1. inference ability (done temporarily)
2. generation ability

### 2. Gaussian mixture model (GMM)

### EM algorithm for training the params for the GMM

In [5]:
from IPython.core.formatters import Dict
def EM_training(mean:np.array,mixing_coeffs:np.array,covs:Dict, dataset_input:np.array):
  '''
  Args:
  mean=
        shape(mean)= (L X dimension_of_the_problem_instance (whether 1D, 2D, 3D and so on))
  mixing_coeffs=
        shape(mixing_coeffs)= (1 X L)
  covs= dict of covariance matrices.
        shape(covs)= L number of entries, one for each of the P_l(x)
  dataset_input: shape would be (num of data set X dimn_of_the_prob)
  '''
  L=np.shape(mean)[0]
  dimn_of_prob=np.shape(dataset_input)[1]
  # define gamma: responsibility
  num_data=np.shape(dataset_input)[0] ### note shape(dataset_input)=(num data X dimn_of_the_problem)
  gamma=np.zeros(shape=(num_data,L))
  num_iters=1 # denoted as T in notes
  ###
  for t in range(0,num_iters):

    ### evaluate the E step
    # update the gamma.
    for idx_over_data in range(0,num_data):
      ### here i need to evaluate the common denominator that will go inside the next loop
      # print("covs[idx_el]")
      # print(covs[0])
      prod_mixing_coeffs_and_gaussian_l=[mixing_coeffs[0,idx_el]*gaussian_2d(x=dataset_input[idx_over_data,:].reshape((1,dimn_of_prob)),mean=mean[idx_el,:].reshape((1,dimn_of_prob)),cov_mat=covs[idx_el]) for idx_el in range(0,L)]
      # print("pil_Pl_fordfnwnf:")
      # print(prod_mixing_coeffs_and_gaussian_l)
      Dr_for_gamma=np.sum(prod_mixing_coeffs_and_gaussian_l)
      gamma[idx_over_data,:]=[prod_mixing_coeffs_and_gaussian_l[k]/Dr_for_gamma for k in range(0,L)]

    print("gamma obtained:");print(gamma)
    ### M-step: Re-estimating the parameters
    N_l=np.sum(gamma,axis=0)# this is the expression sum_{i=1}^{N(all dataset)} gamma_{il} in the lecture notes
    print("N_l is:");print(N_l)# NOTE:: shape of N_l = (L,) (note that this array is 1D)

    ### 1. updating means: the expresion is like a wtd sum of the vectors
    ###
    for idx_el in range(0,L):
      #print("index el is:");print(idx_el)
      temp_vec=np.array(
          [gamma[idx_over_data,idx_el]*dataset_input[idx_over_data,:]
           for idx_over_data in range(0,num_data)]
          )
      #print("temp vec is:");print(temp_vec)
      #update the mean
      # print("np.sum(temp_vec,axis=0):",np.sum(temp_vec,axis=0))
      # print("mean[idx_el,:]=",mean[idx_el,:])
      mean[idx_el,:]=(1./N_l[idx_el])*np.sum(temp_vec,axis=0)
    print("mean matrix after update:");print(mean)

    ### 2. update the covariance matrix:
    ##### formula for updating the cov mat is:
    ##### sigma_lth=1/N_l[idx_el] * sum_{i=1}^{N(all_dataset)} * gamma_{il}*some_outer_product
    for idx_el in range(0,L):
      #print("updating cov matrix, index el is:",idx_el)
      temp_outer_prod_list=np.array(
          [gamma[idx_over_data,idx_el]*(dataset_input[idx_over_data,:]-mean[idx_el,:]).reshape((dimn_of_prob,1)) @
           (dataset_input[idx_over_data,:]-mean[idx_el,:]).reshape((1,dimn_of_prob))
          for idx_over_data in range(0,num_data)
         ]
          )
      #print("temp outer prodcut list is:");print(temp_outer_prod_list)
      #print("summed over:");print(sum(temp_outer_prod_list))
      covs[idx_el]=(1./N_l[idx_el])*sum(temp_outer_prod_list)
    print("updated covriances");print(covs)

    ### 3. update the mixing coeffs:
    for idx_el in range(0,L):
      mixing_coeffs[0,idx_el]=N_l[idx_el]/num_data
    print("updated mixing_coeffs:");print(mixing_coeffs)

    return mean,covs,mixing_coeffs

In [6]:
# an example to check whether the function EM_training working or not.
# L=4, N=3, dim_of_prob=2
mean=np.array([[1.5,1.25],[0.75,1.25],[1.3,1.7],[1.4,1.63]])
print("mean:");print(mean)
mixing_coeffs=np.array([[0.25,0.25,0.25,0.25]])
covs={0:np.array([[0.02,0.003],[0.003,0.1]]),
      1: np.array([[0.06,0.007],[0.007,0.05]]),
      2:np.array([[0.068,0.003],[0.007,0.05]]),
      3:np.array([[0.06,0.007],[0.017,0.078]])}
dataset_input=np.array([[1.78,1],[1.75,1.3],[1.03,1.08]])
checking_em=EM_training(mean=mean,
                        mixing_coeffs=mixing_coeffs,
                        covs=covs,
                        dataset_input=dataset_input)

mean:
[[1.5  1.25]
 [0.75 1.25]
 [1.3  1.7 ]
 [1.4  1.63]]
gamma obtained:
[[9.18297348e-01 2.95287972e-04 6.13499528e-03 7.52723686e-02]
 [6.55581269e-01 5.90691003e-04 8.22543748e-02 2.61573665e-01]
 [1.10155835e-02 8.15573224e-01 3.47270990e-02 1.38684093e-01]]
N_l is:
[1.5848942  0.8164592  0.12311647 0.47553013]
mean matrix after update:
[[1.76237792 1.12464909]
 [1.03079216 1.08013023]
 [1.54840665 1.222996  ]
 [1.54476721 1.18835153]]
updated covriances
{0: array([[ 0.00397132, -0.00194324],
       [-0.00194324,  0.02173501]]), 1: array([[5.77862753e-04, 9.27957540e-05],
       [9.27957540e-05, 3.73140998e-05]]), 2: array([[0.10562853, 0.02870745],
       [0.02870745, 0.0122072 ]]), 3: array([[0.10920867, 0.02185739],
       [0.02185739, 0.01589628]])}
updated mixing_coeffs:
[[0.52829807 0.27215307 0.04103882 0.15851004]]


In [7]:
# a=np.array([[10,20],[1,2],[111,5]])
# np.sum(a,axis=0),np.sum(a,axis=1)

## Kernel density estimation.

Note that the kernel density estimation model is a non-parametric model, hence it does not require any "training" so to speak!


* discusses how to choose an appropriate bandwidth parameter: https://stats.libretexts.org/Bookshelves/Computing_and_Modeling/Supplemental_Modules_(Computing_and_Modeling)/Regression_Analysis/Nonparametric_Inference_-_Kernel_Density_Estimation#:~:text=Contributors-,Introduction%20and%20definition,%E2%88%88%CE%98%E2%8A%82Rd%7D.

1. Squared exponential kernel:
https://www.mathworks.com/help/stats/kernel-covariance-function-options.html

### defining kernel's required for the assingment.

1. Squared exponential kernel
2. Inverse-multi quadratic kernel

### 1. Squared exponential kernel:

In [23]:
a=np.array([[1,2,3]]); b=np.array([[10,20,30]])
norm(a-b)**2, a@a.T+b@b.T-2*a@b.T #obviously!

(1134.0, array([[1134]]))

In [19]:
def squared_exp_kernel(xa:np.array, xb:np.array,bandwidth:float,amplitude:float=1):
  '''
  xa=row vector of shape (1xdimn_of_the_prob)
  xb= row vector of shape (1X dimn_of_the_prob)
  bandwidth: a hyperparameter
  amplitude: overall variance (set to 1 in the lecture, can be changed as well)
  '''
  diff=xa-xb
  kernel=(amplitude**2) *  np.exp( -((norm(diff))**2)/(2*bandwidth) )
  return kernel

In [24]:
def inverse_multi_quadratic_kernel(xa:np.array, xb:np.array,bandwidth:float):
  '''
  xa=row vector of shape (1xdimn_of_the_prob)
  xb= row vector of shape (1X dimn_of_the_prob)
  bandwidth: a hyperparameter
  '''
  diff=xa-xb
  kernel=bandwidth/(bandwidth+ ((norm(diff))**2) )
  return kernel

In [31]:
def kernel_density_estimator(x:np.array, dataset:np.array, bandwidth:float, kernel_fn):
  '''
  x: a row vector (1X dimn of the problem) at which one wants to evaluate the KDE (prob function)
  dataset= an array of shape (N X dimn of the problem)
  bandwidth: hyperparameter
  kernel_fn: choice of the user
  '''
  num_data=np.shape(dataset)[0]
  list_kernel=[kernel_fn(xa=x,xb=dataset[j,:],bandwidth=bandwidth) for j in range(0,num_data)]
  prob=(1./num_data)*sum(list_kernel)## sometimes instead of 1./num_data , we also have 1./(bandwidth X num_data)
  return prob

In [30]:
a=[10.5,-3.2,7.8]
sum(a), 10.5-3.2+7.8, np.sum(a)

(15.1, 15.1, 15.1)

## Implementation of MMD metric:

a good resource for MMD: https://www.onurtunali.com/ml/2019/03/08/maximum-mean-discrepancy-in-machine-learning.html

(implemented using pytorch)